data_loader

In [3]:
import os
from os import walk
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [136]:
OS_PATH="../data"


def format_read_xml(file_path):
    tree=ET.parse(file_path)
    all_users=tree.getroot()#users
    users=[]
    for user in all_users:
        firstName=user[0].text
        telephone_number=user[1].text
        email=user[2].text
        password=user[3].text
        role=user[4].text
        created_at=user[5].text
        children=[]
        for child in user[6]:
            new_child=[child[0].text, child[1].text]
            children.append(new_child)
        users.append([firstName, telephone_number, email, password, role, created_at, children])
    return users


def get_string_between(string, start_char, stop_char):#function that can be usefull
    index_1=string.index(start_char)
    index_2=string.index(stop_char)
    return string[(index_1 + len(start_char)) : index_2]
def format_read_csv(data):#change format to same as stated in xml for ease of transformation and verification
    for index, item in enumerate(data):
        children=[]
        if len(str(item[6]))>3:
            children_str=item[6]
            children_str=children_str.split(',')
            for child_str in children_str:
                name=child_str[:child_str.index('(')]
                age=get_string_between(child_str, '(', ')')
                children.append([name, age])

        data[index][6]=children
    return data


def format_read_json(data):
    # print(data)
    for index, item in enumerate(data):
        data[index][5]=str(item[5])#timestamp to string
        children=[]
        if item[6]!=[]:
            children_list=item[6]
            for child_dict in children_list:
                name=child_dict['name']
                age=child_dict['age']
                children.append([name, age])
        data[index][6]=children
    return data


def get_file_paths(data_folader_path):
    final_paths=[]
    supported_formats=[".csv", ".xml", ".json"]
    w=walk(data_folader_path)
    for (dirpath, dirnames, filenames) in w:
        for filename in filenames:
            for supported_format in supported_formats:
                if supported_format in filename[-5:]:#checcks last substring
                    final_paths.append(dirpath+"/"+filename)
                    break
    return(final_paths)

def get_file_contents(file_path):
    supported_formats=[".csv", ".xml", ".json"]

    if supported_formats[0] in file_path[-5:]:#csv
        return format_read_csv(pd.read_csv(file_path, delimiter=";").to_numpy().tolist())

    if supported_formats[1] in file_path[-5:]:#xml
        return format_read_xml(file_path)

    if supported_formats[2] in file_path[-5:]:#json
        return format_read_json(pd.read_json(file_path).to_numpy().tolist())

def load_data(data_folader_path):
    data_packs=[]
    file_paths=get_file_paths(data_folader_path)
    for file_path in file_paths:
        data_packs.append(get_file_contents(file_path))
    data=[item for file_sublist in data_packs for item in file_sublist]
    return data
data=(load_data(OS_PATH))

file_paths=get_file_paths(OS_PATH)
print(get_file_paths(OS_PATH))


['../data/users_2.xml', '../data\\a/users.json', '../data\\a\\b/users_1.csv', '../data\\a\\b/users_1.xml', '../data\\a\\c/users_2.csv']


In [144]:
import data_loader

In [145]:
data=data_loader.load_data(data_loader.OS_PATH)

In [146]:
print(data)

[['Russell', '+48817730653', 'jwilliams@example.com', '4^8(Oj52C+', 'admin', '2023-05-15 21:57:02', [['Rebecca', '11'], ['Christie', '17']]], ['Dawn', '+48717279856', 'briancollins@example.net', 'R9AjA5nb$!', 'admin', '2023-11-13 01:28:53', [['Andrew', '3'], ['Nicholas', '13']]], ['Bryan', '(48)242024650', 'billy59@example.com', '%ML4fCTmiz', 'user', '2023-06-25 06:56:34', [['Frank', '11']]], ['Joseph', '018820162', 'justin81@example.org', '*0pED9u@8b', 'user', '2023-01-30 18:52:20', [['Wendy', '3'], ['Mark', '2'], ['April', '18']]], ['Sarah', '318506164', 'johnperry@example.com', 'bd6GvDNA!+', 'admin', '2023-09-27 15:27:18', [['Rachel', '8'], ['Bradley', '13']]], ['Teresa', '00522263177', 'fpage@example.net', 'hJ@2Kbg2tF', 'admin', '2023-11-03 00:23:19', []], ['Gregg', '707470796', 'sherrystein@example.com', 'nO6ySyLH&7', 'user', '2023-09-26 13:14:03', []], ['Ashley', '135086990', 'janice79@example.net', 'kj5JYvkv(6', 'user', '2023-04-07 13:01:16', [['Shirley', '2'], ['Martin', '1']]]

entity loader

In [98]:
from entities.User import User
from entities.Child import Child
from DataLoader import DataLoader, get_string_between
from datetime import datetime

def validate_email(email):
    first= email.count('@') == 1
    second = len(email.split('@')[0])>0
    third = len(get_string_between(email, '@', '.'))>0
    dot_sentence_length=len(email.split('.')[-1])
    fourth = email[-dot_sentence_length:].isalnum() and (dot_sentence_length>0 and dot_sentence_length<5)
    if first and second and third and fourth:
        return True
    else:
        return False



data_loader=DataLoader()
data=data_loader.load_data()
final_data=[]
phone_numbers=[]
for index, user_data in enumerate(data):
    if validate_email(user_data[2]):
        if type(user_data[1])==str:
            phone_number_str=user_data[1].replace(" ", "")
            phone_number_start_index=len(phone_number_str)-9
            phone_number=phone_number_str[phone_number_start_index:]
            phone_number=int(phone_number)
            final_data.append([user_data[0], phone_number, user_data[2], user_data[3], user_data[4], datetime.strptime(user_data[5], '%Y-%m-%d %H:%M:%S'), user_data[6], index])
def myKey(s):
    return s[1]
final_data.sort(key=myKey)
old_phone=0
sublist=[]
phones_for_sublist=[]
to_delete=[]
for index, data in enumerate(final_data):
    if data[1] in phones_for_sublist:
        sublist.append(data)
    else:
        def myKey(s):
            return s[5]
        sublist.sort(key=myKey)
        if len(sublist)>1:
            for item in sublist[:-1]:
                to_delete.append(item[7])
        sublist=[data]
        phones_for_sublist.append(data[1])
    old_phone=data[1]

to_delete.sort()
result=[]
def myKey(s):
    return s[7]
final_data.sort(key=myKey)
for index, item in enumerate(final_data):
    if not index in to_delete:
        children=[]
        for item_1 in item[6]:
            child=Child(item_1[0], int(item_1[1]))
            children.append(child)
        user=User(item[0], item[1], item[2], item[3], item[4], item[5], children)
        result.append(user)


In [100]:
len(result)


85

In [44]:
from EntityLoader import EntityLoader
data=EntityLoader().load_entities()

In [56]:
data[2].get_password()

'%ML4fCTmiz'

In [42]:
def print_all_accounts(data):
    print(len(data))

def print_oldest_account(data):
    def myKey(s):
        return s.get_createdAt()
    data_2=data.copy()
    data_2.sort(key=myKey)
    oldest=data_2[0]
    print("name: "+oldest.get_firstName())
    print("email_address: "+oldest.get_email())
    print(f"created_at: {oldest.get_createdAt()}")

def group_children_by_age(data):
    children_list=[]
    for user in data:
        if user.get_children()!=[]:
            for child in user.get_children():
                children_list.append(child)
    def myKey(s):
        return -s.get_age()
    children_list.sort(key=myKey)
    age=children_list[0].get_age()
    counter=0
    counters=[]
    ages=[age]
    for index, child in enumerate(children_list):
        if age==child.get_age():
            counter+=1
        else:
            counters.append(counter)
            age=child.get_age()
            ages.append(child.get_age())
            counter=1
        if index==len(children_list)-1:
            counters.append(counter)
    sort_list=[]
    for index, item in enumerate(counters):
        sort_list.append([item, ages[index]])
    sort_list.sort()
    for counter, age in (sort_list):
        print("age: "+str(age)+", count: "+str(counter))

def print_children(user_data):
    children_list=user_data.get_children()
    if children_list==[]:
        print("No children assigned to this user")
    else:
        def myKey(s):
            return s.get_name()[0]
        children_list.sort(key=myKey)
        for child in children_list:
            print(child.get_name()+", "+str(child.get_age()))

def find_similar_children_by_age(user_data, data):
    ages_to_find=[]
    children=user_data.get_children()
    if children==[]:
        print("No children assigned to this user")
    else:
        for child in children:
            ages_to_find.append(child.get_age())
    result_data=[]
    for user in data:
        children_found=[]
        for child in user.get_children():
            if child.get_age() in ages_to_find:
                children_found.append([child.get_name(), child.get_age()])
        if children_found!=[]:
            result_data.append([user.get_firstName(), user.get_telephone_number(), children_found])
    for result in result_data:
        children_str=""
        children_found=result[2]
        for index, child_data in enumerate(children_found):
            children_str+=(f'{child_data[0]}, {child_data[1]}')
            if index<len(children_found)-1:
                children_str+='; '
        print(result[0]+", "+str(result[1])+": "+children_str)

In [43]:
find_similar_children_by_age(data[0],data)

Russell, 817730653: Rebecca, 11; Christie, 17
Bryan, 242024650: Frank, 11
Theresa, 513112467: Christopher, 11
Michael, 736121560: Michael, 17
Amanda, 208579481: Marie, 17; Susan, 11
Caroline, 617796987: Leonard, 11
Molly, 838059352: Crystal, 11
John, 727231114: Margaret, 17; Gregory, 17
Tanner, 604020303: Mindy , 11
Brenda, 859092651: Courtney , 17
Joan, 232756993: Danielle , 11
Chad, 882294581: Diana , 17; James , 17
Steven, 691250247: Megan, 11
Curtis, 107058738: Shelia , 17
David, 463078584: Dean , 17
Madeline, 441935720: Natalie , 11
Madeline, 441935720: Natalie , 11
